In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap 
from mpl_toolkits.basemap import Basemap, addcyclic
from my_tools import file_dic, plot_dic
from glob import glob
from scipy.stats import ttest_ind

%matplotlib qt

In [38]:
path = '/media/onno/Volume/'
path_to_write = '/media/onno/Volume/fcst_composite/hovmoller/forecast/'

In [3]:
#load RWP property files
rean_env_gen = xr.open_dataset(path +file_dic['envelope']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_v300_gen = xr.open_dataset(path + file_dic['v_wind']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_v300_gen =  rean_v300_gen.assign_coords(longitude=(((rean_v300_gen.longitude + 180) % 360) - 180)).sortby('longitude')

rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})
rean_v300_gen=rean_v300_gen.rename({'longitude':'lon'});rean_v300_gen=rean_v300_gen.rename({'latitude':'lat'})
# rean_env_gen = rean_env_gen.sel(time=[bool(i) for i in rean_env_gen.time.dt.hour%24==0])
# rean_v300_gen = rean_v300_gen.sel(time=[bool(i) for i in rean_v300_gen.time.dt.hour%24==0])

fcst_env_gen = xr.open_dataset(path+file_dic['envelope']['ERA5RF'],decode_times=False).squeeze()
# GFS_env_gen = xr.open_dataset(path+file_dic['envelope']['GFS'],decode_times=False).squeeze()
# ERA5RF_v300_gen = xr.open_dataset(path+'era5rf_v300_wledit2000-10000_latavg_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
# GFS_v300_gen = xr.open_dataset(path+'gefsrf2_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()

# GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
# GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
# GFS_v300_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_v300_gen.time.values]
fcst_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in fcst_env_gen.time.values]
# ERA5RF_v300_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_v300_gen.time.values]

# GFS_env_gen = GFS_env_gen.rename({'lon':'longitude','lat':'latitude'}); GFS_v300_gen = GFS_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'lon':'longitude','lat':'latitude'})#; ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# GFS_env_gen = GFS_env_gen.assign_coords(lon=(((GFS_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_v300_gen = GFS_v300_gen.assign_coords(lon=(((GFS_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
fcst_env_gen = fcst_env_gen.assign_coords(lon=(((fcst_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# ERA5RF_v300_gen = ERA5RF_v300_gen.assign_coords(lon=(((ERA5RF_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_env_gen = GFS_env_gen.sel(lead=GFS_env_gen.lead.values[[bool(i) for i in GFS_env_gen.lead%24==0]])
# GFS_v300_gen = GFS_v300_gen.sel(lead=GFS_v300_gen.lead.values[[bool(i) for i in GFS_v300_gen.lead%24==0]])
# ERA5RF_env_gen = ERA5RF_env_gen.sel(lead=ERA5RF_env_gen.lead.values[[bool(i) for i in ERA5RF_env_gen.lead%24==0]])
# # ERA5RF_v300_gen = ERA5RF_v300_gen.sel(lead=ERA5RF_v300_gen.lead.values[[bool(i) for i in ERA5RF_v300_gen.lead%24==0]])

# GFS_env_gen['lead'] = GFS_env_gen.lead.values//24
# GFS_v300_gen['lead'] = GFS_v300_gen.lead.values//24
# ERA5RF_env_gen['lead'] = ERA5RF_env_gen.lead.values//24
# # ERA5RF_v300_gen['lead'] = ERA5RF_v300_gehttps://nos.nl/n.lead.values//24

# GFS_env_gen = GFS_env_gen.rename({'latitude':'lat','longitude':'lon'})
# GFS_v300_gen = GFS_v300_gen.rename({'latitude':'lat','longitude':'lon'})
# fcst_env_gen = fcst_env_gen.rename({'latitude':'lat','longitude':'lon'})
# # ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'latitude':'lat','longitude':'lon'})





In [40]:
plt.rcParams.update({'hatch.color': 'k'})

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

modelz = ['ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
lead_day = 5
letterz_ax1 = ['a','b','c','d','e','f']
file_diff_pc = '/rawdata/diff_env_lon_{}_{}_lat_{}_{}_pc_lead_day_5_ERA5RF_{}.nc'
file_diff_pw = '/rawdata/diff_env_lon_{}_{}_lat_{}_{}_pw_lead_day_5_ERA5RF_{}.nc'
file_diff_avg = '/rawdata/diff_env_lon_{}_{}_lat_{}_{}_avg_p40_p60_lead_day_5_ERA5RF_{}.nc'
file_rean_pc = '/rawdata/rean_env_lon_{}_{}_lat_{}_{}_pc_lead_day_5_ERA5RF_{}.nc'
file_rean_pw = '/rawdata/rean_env_lon_{}_{}_lat_{}_{}_pw_lead_day_5_ERA5RF_{}.nc'
file_rean_avg = '/rawdata/rean_env_lon_{}_{}_lat_{}_{}_avg_p40_p60_lead_day_5_ERA5RF_{}.nc'

seasonz = ['DJF','JJA']

fig,axz = plt.subplots(2,3,figsize=(9,9))
for j,season in enumerate(seasonz):
    for i,model in enumerate(modelz):
        diffz_env_pw_total = []
        reanz_env_pw_total = []
        len_pw = 0
        for lat_0,lat_1,lon_0,lon_1 in coordinatez[::-1]:
            diff_env_pw = xr.open_dataset(path_to_write+file_diff_pw.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            diff_env_pw = diff_env_pw.assign_coords(lon=(((diff_env_pw.lon-lon_0+180)%360)-180)).sortby('lon')
            diff_env_pw['lead'] = diff_env_pw['lead']/24 - lead_day
            diffz_env_pw_total.append(diff_env_pw)
            rean_env_pw = xr.open_dataset(path_to_write+file_rean_pw.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            rean_env_pw = rean_env_pw.assign_coords(lon=(((rean_env_pw.lon-lon_0+180)%360)-180)).sortby('lon')
            reanz_env_pw_total.append(rean_env_pw)
            len_pw += len(diff_env_pw.date)
        diff_env_pw_total = xr.concat(diffz_env_pw_total,dim='date')
        diff_env_pw_mean = diff_env_pw_total.mean('date')
        rean_env_pw_total = xr.concat(reanz_env_pw_total,dim='date')
        rean_env_pw_mean = rean_env_pw_total.mean('date')        
        
        diffz_env_pc_total = []
        reanz_env_pc_total = []
        len_pc = 0
        for lat_0,lat_1,lon_0,lon_1 in coordinatez[::-1]:
            diff_env_pc = xr.open_dataset(path_to_write+file_diff_pc.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            diff_env_pc = diff_env_pc.assign_coords(lon=(((diff_env_pc.lon-lon_0+180)%360)-180)).sortby('lon')
            diff_env_pc['lead'] = diff_env_pc['lead']/24 - lead_day
            diffz_env_pc_total.append(diff_env_pc)
            rean_env_pc = xr.open_dataset(path_to_write+file_rean_pc.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            rean_env_pc = rean_env_pc.assign_coords(lon=(((rean_env_pc.lon-lon_0+180)%360)-180)).sortby('lon')
            reanz_env_pc_total.append(rean_env_pc)
            len_pc += len(diff_env_pc.date)
        diff_env_pc_total = xr.concat(diffz_env_pc_total,dim='date')
        diff_env_pc_mean = diff_env_pc_total.mean('date')
        rean_env_pc_total = xr.concat(reanz_env_pc_total,dim='date')
        rean_env_pc_mean = rean_env_pc_total.mean('date')
        
        diffz_env_avg_total = []
        reanz_env_avg_total = []        
        len_avg = 0
        for lat_0,lat_1,lon_0,lon_1 in coordinatez[::-1]:
            diff_env_avg = xr.open_dataset(path_to_write+file_diff_avg.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            diff_env_avg = diff_env_avg.assign_coords(lon=(((diff_env_avg.lon-lon_0+180)%360)-180)).sortby('lon')
            diff_env_avg['lead'] = diff_env_avg['lead']/24 - lead_day
            diffz_env_avg_total.append(diff_env_avg)
            rean_env_avg = xr.open_dataset(path_to_write+file_rean_avg.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            rean_env_avg = rean_env_avg.assign_coords(lon=(((rean_env_avg.lon-lon_0+180)%360)-180)).sortby('lon')
            reanz_env_avg_total.append(rean_env_avg)
            len_avg += len(diff_env_avg.date)
        diff_env_avg_total = xr.concat(diffz_env_avg_total,dim='date')
        diff_env_avg_mean = diff_env_avg_total.mean('date')
        rean_env_avg_total = xr.concat(reanz_env_avg_total,dim='date')
        rean_env_avg_mean = rean_env_avg_total.mean('date')
#         p_value_num = ttest_ind(diff_env_pw_total.v.values,diff_env_pc_total.v.values,axis=0)[1]
#         p_value_bool = p_value_num<0.05
        lonz = diff_env_pw_mean.lon.values
        dayz = diff_env_pw_mean.lead.values
        clr_levels_contourf = np.arange(-3,3.1,.25)
        clr_levels_contour = np.arange(8,36.1,2)
        clr_levels_contour2 = np.arange(-3,3.1)
        ax1 = axz[j,0]
        im=ax1.contourf(lonz,dayz,diff_env_pw_mean.v,cmap='bwr',levels=clr_levels_contourf,extend='both')
#         ax1.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax1.contour(lonz,dayz,rean_env_pw_mean.v,cmap='gist_heat_r',levels=clr_levels_contour,extend='both',linewidths=1)
        cs2=ax1.contour(lonz,dayz,diff_env_pw_mean.v,colors='k',levels=clr_levels_contour2,extend='both',
                        linewidths=.75,alpha=.5,linestyles='dashed')
        ax1.clabel(cs,inline=True,fmt='%.0f')
        ax1.set_title('Persistent Warm Events {}'.format(season))
        ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
        ax1.axhline(y=0,color='grey')
        ax1.set_ylabel('Days before/after onset event')
        ax1.set_xlabel('Pseudo Longitude [°]')
        ax1.set_xticks([-120,-60,0,60,120])
#         ax1.tick_params(labelsize=12)
        ax1.set_yticks(np.linspace(-5,5,11))
#         ax1.text(x=0,y=1.1,s=letterz_ax1[0]+')',transform=ax1.transAxes,fontweight='bold',fontsize=17,
#                        verticalalignment='top')
        ax2 = axz[j,1] 
        ax2.contourf(lonz,dayz,diff_env_avg_mean.v,cmap='bwr',levels=clr_levels_contourf,extend='both')
#         ax2.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax2.contour(lonz,dayz,rean_env_avg_mean.v,cmap='gist_heat_r',levels=clr_levels_contour,extend='both',linewidths=1)
        cs2=ax2.contour(lonz,dayz,diff_env_avg_mean.v,colors='k',levels=clr_levels_contour2,extend='both',
                        linewidths=.75,alpha=.5,linestyles='dashed')
        ax2.clabel(cs,inline=True,fmt='%.0f')
        ax2.set_title('Persistent Average Events {}'.format(season))
        ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
        ax2.axhline(y=0,color='grey')
        ax2.set_ylabel('Days before/after onset event')
        ax2.set_xlabel('Pseudo Longitude [°]')
        ax2.set_xticks([-120,-60,0,60,120])
        ax2.set_yticks(np.linspace(-5,5,11))
#         ax2.tick_params(labelsize=12)
#         ax2.text(x=0,y=1.05,s=letterz_ax1[1]+')',transform=ax2.transAxes,fontweight='bold',fontsize=17,
#                        verticalalignment='top')
        ax3 = axz[j,2] 
        ax3.contourf(lonz,dayz,diff_env_pc_mean.v,cmap='bwr',levels=clr_levels_contourf,extend='both')
#         ax3.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax3.contour(lonz,dayz,rean_env_pc_mean.v,cmap='gist_heat_r',levels=clr_levels_contour,extend='both',linewidths=1)
        cs2=ax3.contour(lonz,dayz,diff_env_pc_mean.v,colors='k',levels=clr_levels_contour2,extend='both',
                        linewidths=.75,alpha=.5,linestyles='dashed')
        ax3.clabel(cs,inline=True,fmt='%.0f')
        ax3.set_title('Persistent Cold Events {}'.format(season))
        ax3.axvline(x=0,color='grey');ax3.axvline(x=0,color='grey')
        ax3.axhline(y=0,color='grey')
        ax3.set_ylabel('Days before/after onset event')
        ax3.set_xlabel('Pseudo Longitude [°]')
        ax3.set_xticks([-120,-60,0,60,120])
        ax3.set_yticks(np.linspace(-5,5,11))
#         ax3.tick_params(labelsize=12)
#         ax3.text(x=0,y=1.05,s=letterz_ax1[1]+')',transform=ax3.transAxes,fontweight='bold',fontsize=17,
#                        verticalalignment='top')

for k,ax in enumerate(axz.flat):
    ax.label_outer()
    ax.text(x=-0.05,y=1.066,s=letterz_ax1[k]+')',transform=ax.transAxes,fontsize=14,fontweight='bold',
                       verticalalignment='top')
cbar_ax = fig.add_axes([0.2, 0.05, 0.6, 0.02])
cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
cbar.ax.get_xaxis().set_ticks([-3,-2,-1,0,1,2,3])
cbar.ax.set_xticklabels([-3,-2,-1,0,1,2,3])
cbar.ax.set_xlabel('E (m/s)')
fig.subplots_adjust(left=0.06,right=0.98,bottom=0.13,top=0.96,hspace=0.1,wspace=0.04)
# fig.suptitle("ERA5RF Averaged Hovmöller Persistent Warm, Cold and Average Weather Day 5 Forecast {}".format(season))
fig.savefig('/media/onno/Volume/Main_Figures/composite_hovmollers.png',dpi=250)
        
    